## Logistic Regression

In this section we will implement a logistic regression model trainable with SGD using numpy. Here are the objectives:

1. Implement a simple forward model with no hidden layer (equivalent to logistic regression):
$y = softmax(\mathbf{W} x + b)$

1. build a `predict` function which returns the most probable class given an input $x$

1. build an `accuracy` function for a batch of inputs $X$ and the corresponding expected outputs $y_{true}$

1. understand the method computing $\frac{d}{dW} -\log(softmax(Wx + b))$ for an $x$ and its corresponding expected output $y_{true}$ ; check that the gradients are well defined/finite.

1. build a `train` function which uses the `grad` function output to update $\mathbf{W}$ and $b$

In this exercise we will take small images of hand written digits and classify them. This is a multi-class classification problem. We will use Logistic regression.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14

from sklearn.datasets import load_digits
import numpy as np

digits = load_digits()

In [ ]:
digits.data.shape

For each sample our model will output ten values. The probability it assigns for the image being in each of the ten classes. They should sum to one.

We express our desired output, the ground truth in a similar fashion by one-hot encoding it. So the class 3 will be represented as a vector `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`.

First let's define a helper function to compute the one hot encoding of an integer array for a fixed number of classes:

In [ ]:
def one_hot(n_classes, y):
    return np.eye(n_classes)[y]

In [ ]:
one_hot(10, 3)

In [ ]:
one_hot(10, [3,2,1,0])

In [ ]:
one_hot(10, [0, 4, 9, 1])

Let's take a moment to take a look at the dataset before we start using it.

In [ ]:
sample_index = 45 # change this to see different examples
plt.figure(figsize=(3, 3))
plt.imshow(digits.images[sample_index], cmap=plt.cm.gray_r,
           interpolation='nearest')
plt.title("image label: %d" % digits.target[sample_index]);

### Preprocessing

- normalization (for more take a look at http://scikit-learn.org/stable/modules/preprocessing.html)
- train/test split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


data = np.asarray(digits.data, dtype='float32')
target = np.asarray(digits.target, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.15, random_state=37)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Y_train = one_hot(10, y_train)
Y_test = one_hot(10, y_test)

In [ ]:
Y_train = one_hot(10, y_train)
Y_train[:3]

Let's display the one of the transformed samples (after feature standardization):

In [ ]:
sample_index = 45
plt.figure(figsize=(3, 3))
plt.imshow(X_train[sample_index].reshape(8, 8),
           cmap=plt.cm.gray_r, interpolation='nearest')
plt.title("transformed sample\n(standardised)");

The scaler object makes it possible to recover the original sample:

In [ ]:
plt.figure(figsize=(3, 3))
plt.imshow(scaler.inverse_transform(X_train[sample_index]).reshape(8, 8),
           cmap=plt.cm.gray_r, interpolation='nearest')
plt.title("original sample");

Now let's implement the softmax vector function:

$$
softmax(\mathbf{x}) = \frac{1}{\sum_{i=1}^{n}{e^{x_i}}}
\cdot
\begin{bmatrix}
  e^{x_1}\\\\
  e^{x_2}\\\\
  \vdots\\\\
  e^{x_n}
\end{bmatrix}
$$

In [ ]:
def softmax(X):
    # TODO:
    return None

Make sure that this works one vector at a time (and check that the components sum to one):

In [ ]:
print(softmax([10, 2, -3]))

Note that a naive implementation of softmax might not be able process a batch of activations in a single call (but we need that):

In [ ]:
X = np.array([[10, 2, -3],
              [-1, 5, -20]])
print(softmax(X))

## Loss

This is a multi-class classification problem. The loss function we will use unfortunately has many names that are used interchangably. You can also find many simplified definitions for the two class case that do not note that they are simplifications. For example [wikipedia](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression) has a nice worked example but only a small note pointing out that this is for two classes only. Maybe read this first, then try and extend it to multiple classes using the notation from the beginning of that wikipedia entry.

We will use "log loss", "cross entropy", or even "categorical crossentropy".

The general expression for the log loss of **one sample** is:
$$
\ell = - \sum_k \mathbf{1}_{k=y} log(p_k)
$$

For a problem with $k$ classes, where $y$ denotes the true class, and $p_k$ the probability the network predicts for class $k$. $\mathbf{1}$ is the indicator function, a clever way of saying "zero if `k!=y`, one otherwise". The loss over a complete training (or testing or validation) set of samples is the sum(!) of $\ell$ for each sample.

Implement a function that given the true one-hot encoded class `Y_true` and and some predicted probabilities `Y_pred` returns the negative log likelihood. For this problem we are using the "log loss" loss function (also known as cross-entropy).

This `nll` is a scalar computed using all samples in our training set. The likelihood measures how likely something is. Using the term "likelihood" instead of probability because the likelihood does not have to be a probability.

$$L = P_1 \cdot P_2 \cdot P_3 \cdot ... $$

However multiplying lots of small numbers together is a numerical disaster. Use $\log(ab) = \log(a) + log(b)$ to convert the product into a sum (hence the name log-likelihood). The negative just means we multiply everything by minus one.

In [ ]:
EPSILON = 1e-8 # this is here to give you a hint on how to deal with the case when y_pred=0

def nll(Y_true, Y_pred):
    # TODO
    pass

# Make sure that it works for a simple sample at a time
print(nll([1, 0, 0], [.99, 0.01, 0]))

Check that the `nll` of a very confident yet incorrect prediction is a much higher positive number:

In [ ]:
print(nll([1, 0, 0], [0.01, 0.01, .98]))

Make sure that your implementation can compute the average negative log likelihood of a group of predictions: `Y_pred` and `Y_true` can therefore be passed in as 2D arrays:

In [ ]:
# Check that the average NLL of the following 3 almost perfect
# predictions is close to 0
Y_true = np.array([[0, 1, 0],
                   [1, 0, 0],
                   [0, 0, 1]])

Y_pred = np.array([[0,   1,    0],
                   [.99, 0.01, 0],
                   [0,   0,    1]])

print(nll(Y_true, Y_pred))

Finally we have all the ingredients for training a logistic regression model using gradient descent.

Let's study it **one sample at a time**. To help understand how to implement the various methods here do read on to see how it will be used. Then start implementing things that you can test straight away. For example the forward pass can be tested before trying to train the model. You can also check the computation of the accuracy with a randomly initialised model as you know what its accuracy should be.

In [ ]:
class LogisticRegression():
    def __init__(self, input_size, output_size):
        self.W = np.random.uniform(size=(input_size, output_size),
                                   high=0.1, low=-0.1)
        self.b = np.random.uniform(size=output_size,
                                   high=0.1, low=-0.1)
        self.output_size = output_size
        
    def forward(self, X):
        # TODO: compute normalised scores for each class, this is the output
        # of the softmax()
        pass

    def predict(self, X):
        # TODO: for each sample in X return the predicted class
        # translate the class probabilities of each sample into a human
        # friendly representation using integers
        pass
    
    def grad_loss(self, x, y_true):
        # TODO: compute gradient with respect to W and b for a sample x
        # and the true label y_true
        # If you are confident in your linear algebra skills attempt to
        # write this from scratch.
        y_pred = self.forward(x)

        dnll_output =  y_pred - one_hot(self.output_size, y_true)
        grad_W = np.outer(x, dnll_output)
        grad_b = dnll_output
        grads = {"W": grad_W, "b": grad_b}
        return grads
    
    def train(self, x, y, learning_rate):
        # TODO: compute one step of traditional gradient descent update without momentum
        # and update W and b
        grads = self.grad_loss(x, y)
        # ... do something with `grads`
        
    def loss(self, x, y):
        # TODO: use `nll` to compute the loss for the sample x with true label y
        pass

    def accuracy(self, X, y):
        # TODO: compute accuracy for samples X with true labels y
        pass

In [ ]:
# Build a model and test its forward inference
n_features = X_train.shape[1]
n_classes = Y_train.shape[1]
lr = LogisticRegression(n_features, n_classes)

print("Evaluation of the untrained model:")
train_loss = lr.loss(X_train, y_train)
train_acc = lr.accuracy(X_train, y_train)
test_acc = lr.accuracy(X_train, y_train)

print("train loss: %0.4f, train acc: %0.3f, test acc: %0.3f"
      % (train_loss, train_acc, test_acc))
# Question: do you think the accuracy makes sense?

In [ ]:
# Test the untrained model on an example
sample_idx = 3
plt.plot(lr.forward(X_train[sample_idx]), linestyle='-', label='prediction')
plt.plot(one_hot(10, y_train[sample_idx]), linestyle='--', label='true')
plt.title('output probabilities')
plt.legend()
print(lr.predict(X_train[sample_idx]))

In [ ]:
# Training for one epoch
learning_rate = 0.01

for i, (x, y) in enumerate(zip(X_train, y_train)):
    lr.train(x, y, learning_rate)
    if i % 100 == 0:
        train_loss = lr.loss(X_train, y_train)
        train_acc = lr.accuracy(X_train, y_train)
        test_acc = lr.accuracy(X_test, y_test)
        print("Update #%d, train loss: %0.4f, train acc: %0.3f, test acc: %0.3f"
              % (i, train_loss, train_acc, test_acc))

In [ ]:
# Evaluate the trained model on an example
sample_idx = 899
plt.plot(lr.forward(X_train[sample_idx]), linestyle='-', label='prediction')
plt.plot(one_hot(10, y_train[sample_idx]), linestyle='--', label='true')
plt.title('output probabilities')
plt.legend()
print(lr.predict(X_train[sample_idx]))

Questions:

* can you find examples that are mispredicted, is there a pattern to the wrong predictions?
* visualise these samples and their predicted classes
* plot the [confusion matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) to find classes that are hard to separate (maybe eight vs nine?)
* do things improve with training for more epochs?
* experiment with different values of the learning rate. Can you accelerate learning? What happens for very large values of the learning rate? Very small ones?
* convert the training setup to use stochastic gradient descent, does this change things?
* what is the optimal number of epochs to train for? Why?